# **Web Scraping | SQL**

### **SET-UP**

In [13]:
# import the libraries
import pandas as pd
import json
import datetime
import requests
import sqlite3
from bs4 import BeautifulSoup

**Web Scraping:** Job offers' Website (Luxembourgish)
https://en.jobs.lu/ 

**The objective:** The goal is to parse the HTLM of the site in order to identify and separate the most relevant information. This output will be transformed into a JSON and all data will be stored in BD SQL.

**Filter:** IT / Programming

In [14]:
# Setting-up the environment
def get_data(url, parse):
    raw_html = requests.get(url, parse).content
    return BeautifulSoup(raw_html)

# Identifying the path
url = 'https://en.jobs.lu/'
raw_html = requests.get(url).content
html = BeautifulSoup(raw_html) # Ler HTLM e outros formatos bem como gerar árvore de dados

# Extraction of the information
html = get_data('https://en.jobs.lu/it_programming_jobs.aspx', 'html.parse')
job_offers = html.find_all("article", class_="job-list-item")

# Creating dictionary
my_jobs = {}
for index in range(len(job_offers)):
    job_n = "job_{}".format(index + 1) 
    
    # Pré-process and data cleaning
    today1 = datetime.datetime.today().strftime('%d/%m/%Y') # Conversão de forma a apresentar o dia de cada publicação em formato data (dia da publicação em vez de "Today")
    yesterday1 = datetime.datetime.today() - datetime.timedelta(days=1) # Conversão para datetime de forma a apresentar o dia de cada publicação (cont.)
    yesterday1 = yesterday1.strftime('%d/%m/%Y') # (cont.) em formato data (dia anterior em vez de "yesterday")
    my_jobs[job_n] = {
    "Date" : job_offers[index].find_all("span", class_ = "date")[0].get_text().strip().replace("Today", today1).replace("1 Day Ago", yesterday1),
    "Job" : job_offers[index].find_all("span", class_ = "job")[0].get_text().strip('\n'),
    "Company" : job_offers[index].find_all("span", class_ = "company")[0].get_text().strip('\n'),
    "Location" : str(job_offers[index].find_all("span", class_ = "location")[0].get_text().replace(","," ").rstrip("LU").lstrip('1234567890_F-').strip().capitalize())
    }

In [6]:
#my_jobs
df_jobs = pd.DataFrame(my_jobs).T # Displaying results in a dataframe
df_jobs # Calling dataframe

,Date,Job,Company,Location
job_1,25/08/2020,Life Cycle Assessment Analyst with proven Prog...,Luxembourg Institute of Science and Technology...,Belval
job_2,25/08/2020,Ingénieur DevOps (m/f),Bâloise Assurance Luxembourg,Bertrange luxembourg
job_3,25/08/2020,Junior Systems and Storage Technician (m/f),BCE Broadcasting Center Europe S.A.,L1543 luxembourg
job_4,25/08/2020,IT Admin System Manager,Link Point SA,Luxembourg
job_5,25/08/2020,Microsoft SQL Server Administrator (f/m/d),Dennemeyer,Howald
job_6,25/08/2020,Project Manager (f/m/d),Dennemeyer,Howald
job_7,25/08/2020,Fullstack Developer (f/m/d),Dennemeyer,Howald
job_8,25/08/2020,(Junior) Data Integration Specialist (f/m/d),Dennemeyer,Howald
job_9,25/08/2020,Infrastructure Systems and Solutions Specialis...,Dennemeyer,Howald
job_10,25/08/2020,IT Compliance Manager (m/f) IT-COM,Commission de Surveillance du Secteur Financie...,Luxembourg


# **SQL**
#### Storage

In [7]:
# Creation of the BD file, creation of the table, configuration and categorization of attributes
db_file = "it_jobs.sqlite.db" 
conn = sqlite3.connect(db_file) # Starts connection
cur = conn.cursor() # Cursor definition that allows manipulating and navigating with the DB
# Creating table
query = """ CREATE TABLE IF NOT EXISTS jobsit ( 
                                    date datetime,
                                    job text,
                                    company text,
                                    location text
                                ); """
cur.execute(query) # Creating table, runs the query

In [8]:
conn.commit() # Recording the table in the database
df_jobs = pd.DataFrame(my_jobs).T # Creating dataframe
df_jobs.to_sql("jobsit", con=conn, if_exists="append", index=False) # Setting parameters and filling the table with data
df_jobs

,Date,Job,Company,Location
job_1,25/08/2020,Life Cycle Assessment Analyst with proven Prog...,Luxembourg Institute of Science and Technology...,Belval
job_2,25/08/2020,Ingénieur DevOps (m/f),Bâloise Assurance Luxembourg,Bertrange luxembourg
job_3,25/08/2020,Junior Systems and Storage Technician (m/f),BCE Broadcasting Center Europe S.A.,L1543 luxembourg
job_4,25/08/2020,IT Admin System Manager,Link Point SA,Luxembourg
job_5,25/08/2020,Microsoft SQL Server Administrator (f/m/d),Dennemeyer,Howald
job_6,25/08/2020,Project Manager (f/m/d),Dennemeyer,Howald
job_7,25/08/2020,Fullstack Developer (f/m/d),Dennemeyer,Howald
job_8,25/08/2020,(Junior) Data Integration Specialist (f/m/d),Dennemeyer,Howald
job_9,25/08/2020,Infrastructure Systems and Solutions Specialis...,Dennemeyer,Howald
job_10,25/08/2020,IT Compliance Manager (m/f) IT-COM,Commission de Surveillance du Secteur Financie...,Luxembourg


In [9]:
conn.close() # Disconnect db connection

In [10]:
conn = sqlite3.connect(db_file) # Starts connection
cur = conn.cursor() # Cursor definition that allows manipulating and navigating with the DB
query1 = """
    SELECT * 
    FROM jobsit
    WHERE location="Luxembourg"
    OR company="FYTE"
"""
cur.execute(query1) # Runs the query
call = cur.fetchall() # Getting results
show = pd.DataFrame(call) # Creates DataFrame
show # Shows DataFrame

,0,1,2,3
0,30/03/2020,Information Security Officer (H/F) en CDI,AXA Assurances Luxembourg,Luxembourg
1,30/03/2020,Senior Cloud Architect (Azure),The Recruiter Sàrl,Luxembourg
2,30/03/2020,urgent ! .Net developpeur WPF,Fast Recruitment,Luxembourg
3,30/03/2020,IT Junior Trainer and Support Analyst (m/f),Clifford Chance,Luxembourg
4,30/03/2020,System Engineer - VMware,The Recruiter Sàrl,Luxembourg
...,...,...,...,...
60,24/08/2020,Développeur confirmé PHP/symfony fullstack (H/F),CockpitLAB,Luxembourg
61,24/08/2020,Data Base Administrator (m/f/d),European Depositary Bank SA,Luxembourg
62,24/08/2020,IT Technical Domain Expert / Senior System Eng...,Bank GPB International S.A.,Luxembourg
63,23 Aug,Webmaster (H/F),Randstad Luxembourg,Luxembourg


In [11]:
conn.close() # Disconnect db connection

In [12]:
show

,0,1,2,3
0,30/03/2020,Information Security Officer (H/F) en CDI,AXA Assurances Luxembourg,Luxembourg
1,30/03/2020,Senior Cloud Architect (Azure),The Recruiter Sàrl,Luxembourg
2,30/03/2020,urgent ! .Net developpeur WPF,Fast Recruitment,Luxembourg
3,30/03/2020,IT Junior Trainer and Support Analyst (m/f),Clifford Chance,Luxembourg
4,30/03/2020,System Engineer - VMware,The Recruiter Sàrl,Luxembourg
...,...,...,...,...
60,24/08/2020,Développeur confirmé PHP/symfony fullstack (H/F),CockpitLAB,Luxembourg
61,24/08/2020,Data Base Administrator (m/f/d),European Depositary Bank SA,Luxembourg
62,24/08/2020,IT Technical Domain Expert / Senior System Eng...,Bank GPB International S.A.,Luxembourg
63,23 Aug,Webmaster (H/F),Randstad Luxembourg,Luxembourg


---

**Author**: Gonçalo Guimarães Gomes. Portuguese. Data Analyst and Digital Brand Marketer. Postgraduate in Data Science and in Digital Marketing. Degree and Executive Master in Marketing Management.

### **Contacts**

- [Linkedin](https://www.linkedin.com/in/goncaloggomes/)
- [Twitter](https://twitter.com/goncaloggomes)
- [Medium Profile](https://medium.com/@goncaloggomes)
- [GitHub](https://github.com/goncaloggomes)
- [Email](mailto:goncaloggomes@gmail.com)